In [1]:
from utils.data_cleaning import run_listen_pipeline
from utils.data_preprocess import codic_of_index, train_test_split
from utils.evaluate import evaluate_model
from utils.cf_utils import *
from utils.models import *

import pandas as pd
import polars as pl


import faiss
from tqdm import tqdm
from sklearn.pipeline import Pipeline


/opt/venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Data

### DataLoding

In [2]:
data = pd.read_parquet("data/multi_event.parquet", engine="fastparquet")
items_meta = pd.read_parquet("data/items_meta.parquet")

### Train/test split

Для начала разделим данные на обучающую и тестовую выборки. Деление выполняется по временному признаку, поскольку мы работаем с последовательностями действий пользователей.

Чтобы избежать data leakage — ситуации, когда признаки, рассчитанные на исторических данных, могут «заглядывать» в будущее — вводится временной зазор (gap) между обучающей и тестовой частями.

В оригинальной статье https://www.arxiv.org/pdf/2505.22238  используется соотношение 300 дней обучения, 30 минут gap, 1 день теста.

В моих экспериментах наилучшие результаты получились при конфигурации: 80 дней обучения, 30 минут gap, 20 дней теста.

Чтобы приблизиться к условиям статьи, я планирую снижать размер тестовой выборки, сохраняя возможность варьировать объём обучающей части как гиперпараметр модели. Такой подход позволяет гибко контролировать баланс между длиной обучающего периода и реалистичностью оценки качества.

In [3]:
train_df, test_df = train_test_split(data, 1, 40, gap_size = 30)

### Очиска и кодирование данных 
Применим необходимую предобработку для очистки и подготовки данных. Подробные рассуждения о шагах и мотивации предобработки приведены в блокноте multi_event_preprocessing.
Все функции вынесены в отдельный модуль, что делает код более читаемым, модульным и удобным для повторного использования.

Кодируем uid и item_id в числовые индексы, чтобы модели и алгоритмы могли с ними работать.

Строковые ID нельзя напрямую подать в матрицу взаимодействий или Embedding-слой, поэтому мы переводим их в диапазон 0..N.
Это нужно для эффективности (память, скорость), совместимости с моделями (Matrix Factorization, Embeddings) и стабильного маппинга между train и test.

In [4]:
print(len(test_df))
print(len(train_df))

train_df, test_df , item_id_map = codic_of_index(train_df, test_df)

train_df_likes = train_df[~(train_df["event_type"] == "listen")]
test_df_likes = test_df[~(test_df["event_type"] == "listen")]

train_df_listen = train_df[train_df["event_type"] == "listen"]
test_df_listen = test_df[test_df["event_type"] == "listen"]


train_df_listen, stats = run_listen_pipeline(train_df_listen,
                                            ["remove_duplicates_by_timestamps", 
                                             # "remove_not_valid_listen_data", 
                                             "cut_track_len", 
                                             "remove_rer_data"])


test_df_listen, stats = run_listen_pipeline(test_df_listen,
                                            ["clean_listen_duplicates", 
                                             # "remove_not_valid_listen_data",
                                            ])


train_df_likes, stats = run_listen_pipeline(train_df_likes,
                                            ["convert_reaction", 
                                             "rename_events"])


test_df_likes, stats = run_listen_pipeline(test_df_likes,
                                            ["convert_reaction", 
                                             "rename_events"])


test_df = pd.concat([test_df_likes,test_df_listen ]) 
train_df = pd.concat([train_df_likes,train_df_listen ]) 

items_meta = pd.read_parquet("data/items_meta.parquet")
items_meta["item_id"] = items_meta["item_id"].map(item_id_map)
items_meta = items_meta.drop_duplicates(subset=["item_id"])
items_meta = items_meta.dropna()


df_merged = train_df.merge(items_meta[['item_id', 'artist_id', 'album_id']], 
                      on='item_id', 
                      how='left')

print(len(test_df))
print(len(train_df))

238398
8509047
216905
5826484


In [5]:
test_df_likes = None
train_df_likes = None
train_df_listen= None
test_df_listen= None

# Models

## Baseline

#### Глобальные тренды

Самый простой базовый подход — взять топ-10 самых популярных треков. При этом возможны два варианта отбора:

* по количеству лайков,
* по количеству прослушиваний.

Реализованы обе версии, однако в качестве базовой модели (baseline) выбран именно топ-10 самых залайканных треков среди всех пользователей. Лайки считаются более надёжным сигналом предпочтений, чем прослушивания. (Для чистоты эксперимента был протестирован и вариант с прослушиваниями — его точность оказалась ниже.)



Поскольку тренды и популярные песни сменяются очень быстро, временное окно, в котором мы определяем тренд, также должно быть коротким. (*PS нужно оптимизировать выбор автоматически*)

!!! Важно отметить что в каждомй модели внутри стоит фильтрация на песни, которые пользователь уже видел.

In [ ]:
model = MostPop_by_likes()
last_5days = train_df[train_df['timestamp'] > (train_df['timestamp'].max() - 5 * DAY_SECONDS)]
model.fit(last_5days)

evaluate_model(model, train_df, test_df)

 25%|███████████████▎                                             | 1254/4982 [00:07<00:27, 136.91it/s]

In [7]:
model = MostPop_by_listen()
model.fit(last_5days)

evaluate_model(model, train_df, test_df)

100%|██████████████████████████████████████████████████████████████| 4982/4982 [01:19<00:00, 62.87it/s]

Mean Recall@10: 0.005743334642515487
Mean NDCG@10: 0.013812756478449316


#### Популярные треки любимых артистов 

Модель подбирает треки от любимых артистов пользователя, которые он ещё не слушал. Она анализирует предпочтения по артистам и предлагает как новые, так и ранее пропущенные композиции, сохраняя рекомендации в рамках привычного музыкального вкуса пользователя.

In [8]:
model = UnheardFavoritesRecommender()
model.fit(df_merged)

evaluate_model(model, train_df, test_df)

100%|█████████████████████████████████████████████████████████████████████████████████| 4972/4972 [01:21<00:00, 60.70it/s]

Mean Recall@10: 0.005371003869557287
Mean NDCG@10: 0.016528482337907945


#### Новинки последних дней

Похоже на модель популярных песен, но берет только новые песни. В топ прослушаных могут попадать к примеру песня Шуфутинского 3го сентября 

In [9]:
model = NewItemsLastNDays()
model.fit(train_df)

evaluate_model(model, train_df, test_df)

100%|████████████████████████████████████████████████████████████████████████████████| 4972/4972 [00:27<00:00, 183.28it/s]

Mean Recall@10: 0.0052844955209386
Mean NDCG@10: 0.016498666560238358


#### Новинки любимых артистов 

In [10]:
model = FavoriteArtistsNewReleases()
model.fit(df_merged)

evaluate_model(model, train_df, test_df)

100%|█████████████████████████████████████████████████████████████████████████████████| 4972/4972 [04:45<00:00, 17.42it/s]

Mean Recall@10: 0.005323613635471547
Mean NDCG@10: 0.01683684717428109


## Collaborative filtering

### ALS

AlternatingLeastSquares (ALS) — это модель матричной факторизации для неявных откликов (implicit feedback).
Она раскладывает матрицу взаимодействий пользователей и объектов на две матрицы эмбеддингов — пользователей и айтемов — так, чтобы их скалярное произведение предсказывало интерес.
Обучается поочерёдным решением задач наименьших квадратов: сначала фиксирует айтемы и обновляет пользователей, потом наоборот.


В одной итерации ALS поочерёдно пересчитывает вектора пользователей и айтемов.
Сначала, при фиксированных айтемах, решаются задачи наименьших квадратов для всех пользователей;
потом, при фиксированных пользователях — для всех айтемов.
Это повторяется несколько итераций, пока модель не сойдётся.

Для обучения ALS требуется подготовить данные в формате матрицы взаимодействий, где каждая ячейка отражает степень интереса пользователя к объекту. Поскольку в нашем случае используются implicit-сигналы, нам необходимо преобразовать исходные события (прослушивание, лайк, дизлайк) в числовые веса.

На этапе подготовки данных мы подсчитываем, сколько раз пользователь прослушал каждый трек, а также приводим реакции к бинарному виду:
в результирующем датафрейме поле like получает значение 1,
аналогично dislike — значение 1.

В отдельном модуле реализована функция объединения данных, которая формирует итоговый датафрейм с колонками:

* признак лайка (like = 1)
* признак дизлайка (dislike = 1)
* количество прослушиваний каждой пары пользователь–трек

In [8]:
train_df_als = merge_data_by_count(train_df)

# ToDo формулу описать!!! зачем логорифирование и описать смысл коэффициентов  
train_df_als["conf"] = (train_df_als["have_like"] * 20.0) + (train_df_als["have_dislike"] * -10) + np.log1p(train_df_als["listen_count"]*(train_df_als["played_ratio_pct"]/100))


Для ALS необходима разреженная матрица вида:

(user_id, item_id) → weight


где weight — это итоговая сила взаимодействия (прослушивания + реакции).
В модуле реализована функция, которая на основе сформированного датафрейма строит такую матрицу через coo_matrix и возвращает её в подходящем для модели формате.

In [9]:
matrix = create_user_item_matrix(train_df_als)
matrix

<Compressed Sparse Row sparse matrix of dtype 'int32'
	with 2450831 stored elements and shape (9954, 376570)>

In [10]:
model = ALS()

model.model = AlternatingLeastSquares(factors=128, #Размерность скрытых признаков (эмбеддингов)
                                regularization=0.05,
                                iterations=30,
                                alpha=1,
                                random_state=42)

model.fit(matrix)

/opt/venv/lib/python3.11/site-packages/implicit/cpu/als.py:95: RuntimeWarning: OpenBLAS is configured to use 16 threads. It is highly recommended to disable its internal threadpool by setting the environment variable 'OPENBLAS_NUM_THREADS=1' or by calling 'threadpoolctl.threadpool_limits(1, "blas")'. Having OpenBLAS use a threadpool can lead to severe performance issues here.
  check_blas_config()
100%|██████████████████████████████████████████████████████████████████| 30/30 [00:20<00:00,  1.43it/s]


In [11]:
evaluate_model(model, train_df, test_df)

100%|██████████████████████████████████████████████████████████████| 4974/4974 [02:44<00:00, 30.15it/s]


Mean Recall@10: 0.008001553961338555
Mean NDCG@10: 0.023238393336016756


Метод ALS работает только с пользователями и объектами, которые присутствовали в обучающей выборке. Если в тесте встречается пользователь, которого модель ранее не видела, для него невозможно построить профиль в факторизационном пространстве.

В такой ситуации ALS не способен выдать рекомендации — в классе ALS такая логика приводит к возврату пустого списка.

Поэтому при использовании ALS важно учитывать, что:

* рекомендации гарантированно работают только для пользователей из train,
* все новые или ранее невстречавшиеся пользователи будут давать «пустой» результат.

Это естественное ограничение моделей матричной факторизации, основанных на implicit-feedback данных.

### ItemItemRec

In [8]:
model = ItemItemRec()
model.fit(matrix)

evaluate_model(model, train_df, test_df)

100%|████████████████████████████████████████████████████████████████████████████████| 4972/4972 [00:29<00:00, 169.25it/s]

Mean Recall@10: 0.006586899652643295
Mean NDCG@10: 0.017713902007429942


У авторов в статье точность была намного выше - возможно из-за того, что обучались на всем наборе данных, а не на одном месяце

### BPR

In [14]:
# BPR ожидает ITEM×USER матрицу, поэтому транспонируем
# (это стандарт для implicit: матрица item-user)

item_user_matrix = matrix.T.tocsr()

model = BPR()
model.fit(matrix)

evaluate_model(model, train_df, test_df)

100%|█████████████████████████████████████████████████████████████████████████████████| 4972/4972 [02:39<00:00, 31.18it/s]

Mean Recall@10: 0.00763476834516413
Mean NDCG@10: 0.021159870564358627


## Content_based 

Content-Based Filtering — это метод рекомендаций, который подбирает объекты, похожие на те, что пользователь уже оценил, используя их содержательные признаки (жанры, embedding-и, особенности контента). Его сильная сторона в том, что он не зависит от других пользователей, хорошо работает для новых объектов (нет cold-start для items) и даёт легко объяснимые рекомендации. Однако у подхода есть ограничения: качество сильно зависит от самих признаков, модель не умеет учитывать тренды или контекст поведения, а рекомендации могут быть слишком однообразными и «сужать» вкусы пользователя.


#### embeddongs 

In [6]:
train_df_als = merge_data_by_count(train_df)
content_based_train = train_df_als[(train_df_als["have_dislike"]==0)][(train_df_als["listen_count"]>5) | (train_df_als["have_like"]==1)]


/tmp/ipykernel_84557/1594391976.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  content_based_train = train_df_als[(train_df_als["have_dislike"]==0)][(train_df_als["listen_count"]>5) | (train_df_als["have_like"]==1)]


Методы точного поиска ближайших соседей (kNN) требуют вычислять расстояние между вектором-запросом и каждым объектом в базе, поэтому их сложность линейная 
𝑂
(
𝑁
)
O(N), и на больших наборах данных с десятками или сотнями тысяч элементов они становятся крайне медленными: нужно выполнить миллионы скалярных произведений при каждом запросе. По этой причине в рекомендациях и поиске похожих объектов чаще используют ANN (Approximate Nearest Neighbors) — алгоритмы, которые строят специальный индекс (HNSW, Annoy, Faiss и др.) и ищут ближайших соседей не полностью перебирая весь набор, а умно обходя лишь небольшую часть точек. В результате скорость поиска ускоряется в десятки–сотни раз при минимальной потере качества, что делает ANN практически стандартом для больших эмбеддингов.

In [7]:
index, all_item_ids = create_index("data/filtered_embeddings.parquet", item_id_map)

added up to offset=100000, total indexed=41811
added up to offset=150000, total indexed=62931
added up to offset=200000, total indexed=83899
added up to offset=250000, total indexed=104823
added up to offset=300000, total indexed=125712
added up to offset=350000, total indexed=146699
added up to offset=400000, total indexed=167787
added up to offset=450000, total indexed=188867
added up to offset=500000, total indexed=209730
added up to offset=550000, total indexed=230634
added up to offset=600000, total indexed=251553
added up to offset=650000, total indexed=272647
added up to offset=700000, total indexed=293705
added up to offset=750000, total indexed=314577
added up to offset=800000, total indexed=335563
added up to offset=850000, total indexed=355573


In [14]:
model = CBF_by_embeding()
model.fit(content_based_train, index, all_item_ids)

In [15]:
evaluate_model(model, train_df, test_df)

100%|█████████████████████████████████████████████████████████████████████████████████| 4972/4972 [01:08<00:00, 72.23it/s]

Mean Recall@10: 0.0013124998004842466
Mean NDCG@10: 0.003722185476880808


## Ранжирование

Ранжирование (Learning to Rank) — это задача, в которой модель не просто предсказывает значение, а упорядочивает объекты так, чтобы наиболее релевантные оказались выше в списке. Такой подход используется в поиске, рекомендациях и подборе контента — везде, где важна правильная сортировка результатов.

CatBoost хорошо подходит для ранжирования, потому что:

* поддерживает современные алгоритмы LTR (YetiRank, PairLogit и др.);
* отлично работает с категориальными признаками без дополнительной обработки;
* показывает высокую устойчивость и качество «из коробки»;
* легко использовать — достаточно передать группы (group_id).

Поэтому CatBoostRanker — удобная и мощная модель для построения систем рекомендаций и поиска.

### Гибридная модель

Объединием все полученные модели в одну гибридную. Каждая будет выдвать свой топ предсказаний ( но будем брать не 10 а больше). Затем полученные результаты будут уходить на вход модели ранжирования

In [6]:
from catboost import CatBoostRanker
from utils.evaluate import filtering_listened_items


# model = FavoriteArtistsNewReleases()
# model.fit(df_merged)

# evaluate_model(model, train_df, test_df)


# model = UnheardFavoritesRecommender()
# model.fit(df_merged)

# evaluate_model(model, train_df, test_df)



# evaluate_model(model, train_df, test_df)
# model = CBF_by_embeding()
# model.fit(content_based_train, index, all_item_ids)


class HybridModel:
    def __init__(self):
        self.trend_model = MostPop_by_likes()
        self.new_items_model = NewItemsLastNDays()
        self.als = ALS()

        self.favorite_artists_new_releases_model = FavoriteArtistsNewReleases()
        self.unheard_favorites_recommender_model = UnheardFavoritesRecommender()

        self.N = 10

        self.limit = 50

    
    def fit(self, df, df_merged, conf_train_data):
        print("Prepere data")
        self.history = df
        last_5days = df[df['timestamp'] > (df['timestamp'].max() - 5 * DAY_SECONDS)]
        # last_20days = df_merged[df_merged['timestamp'] > (df_merged['timestamp'].max() - 20 * DAY_SECONDS)]

        user_item_relevance = (
            conf_train_data
            .set_index(["uid", "item_id"])["conf"]
            .to_dict()
        )

        hybrid_train_users = conf_train_data["uid"].unique().tolist()

        matrix = create_user_item_matrix(conf_train_data)

        
        print("Train recomended models")
        print("trend_model")
        self.trend_model.fit(last_5days)
        self.new_items_model.fit(df)
        self.favorite_artists_new_releases_model.fit(df_merged)
        self.unheard_favorites_recommender_model.fit(df_merged)
        
        self.als.fit(matrix)

        
        print("Prepere data for rancin model")
        
        X_rows = []      # сюда складываем фичи (строки)
        y_train = []     # сюда релевантность
        group_id = []    # группы (по пользователю)
        
        current_group = 0

        for user_id in tqdm(hybrid_train_users):
            # кандидаты от гибридной системы
            candidates = self.gather_candidates(user_id)
            
            if not candidates:
                # если модель не смогла ничего отдать – пропускаем пользователя
                continue
        
            rows = self.make_features(user_id, candidates)
            if not rows:
                continue

            n_rows = len(rows)
        
            for row in rows:
                item_id = row["item_id"]
        
                rel = user_item_relevance.get((user_id, item_id), 0.0)
                # print(rel)
                X_rows.append(row)
                y_train.append(rel)
        
            group_id.extend([current_group] * n_rows)
            current_group += 1
        
        self.X_train = pd.DataFrame(X_rows)
        self.y_train = y_train
        self.group_id = group_id
        
        print("Train rancin model")
        
        self.hybrid_model = CatBoostRanker(
            iterations=300,
            learning_rate=0.05,
            depth=6,
            loss_function="YetiRank",
            verbose=50,
        )
        
    def fit_ranker(self):
        self.hybrid_model.fit(
            self.X_train,
            self.y_train,
            group_id=self.group_id,
        )


    def gather_candidates(self, uid):

        candidates = {}
        
        rec, weights = self.trend_model.recommend(uid)
        rec, weights = rec[:self.limit], weights[:self.limit]
    
        for item, score in zip(rec, weights):
            candidates.setdefault(item, {})['trend'] = score
    
        rec, weights = self.new_items_model.recommend(uid)
        rec, weights = rec[:self.limit], weights[:self.limit]
    
        for item, score in zip(rec, weights):
            candidates.setdefault(item, {})['new'] = score

        rec, weights = self.als.recommend(uid)
        rec, weights = rec[:self.limit], weights[:self.limit]
    
        for item, score in zip(rec, weights):
            candidates.setdefault(item, {})['als'] = score

        rec, weights = self.favorite_artists_new_releases_model.recommend(uid)
        rec, weights = rec[:self.limit], weights[:self.limit]
    
        for item, score in zip(rec, weights):
            candidates.setdefault(item, {})['favorite_artists_new_releases_model'] = score

        rec, weights = self.unheard_favorites_recommender_model.recommend(uid)
        rec, weights = rec[:self.limit], weights[:self.limit]
    
        for item, score in zip(rec, weights):
            candidates.setdefault(item, {})['unheard_favorites_recommender_model'] = score
            
        return candidates

    def make_features(self, user_id, candidates):
        rows = []
        for item_id, scores in candidates.items():
    
            row = {
                "user_id": user_id,
                "item_id": item_id,
        
                "score_trend":      scores.get("trend", 0),
                "favorite_artists_new_releases_model":   scores.get("favorite_artists_new_releases_model", 0),
                "score_als":   scores.get("als", 0),
                "unheard_favorites_recommender_model":   scores.get("unheard_favorites_recommender_model", 0),
                "score_new":   scores.get("new", 0),
            }
    
            rows.append(row)
        
        return rows
            
    def recommend(self, uid):
        
        """
        Инференс мета-ранжировщика:
        - hybrid.gather_candidates(user_id) — отдает кандидатов от базовых моделей
        - make_features(user_id, candidates) — строит те же фичи, что и на обучении
        - model.predict(...) — CatBoostRanker, обученный ранее
        """
    

        candidates = self.gather_candidates(uid)
        if not candidates:
           
            return []
    
        rows = self.make_features(uid, candidates)
        if not rows:
            return []
    
        df = pd.DataFrame(rows)

        user_listened_items = get_user_listened_items(self.history, uid)
        
        df = df[~df["item_id"].isin(user_listened_items)]
    
        scores = self.hybrid_model.predict(df)
        df["score"] = scores
    
        df_sorted = df.sort_values("score", ascending=False)
        
        return df_sorted["item_id"].tolist(), df_sorted["score"].tolist()  


In [7]:
import numpy as np
import pandas as pd

train_df_als = merge_data_by_count(train_df)

train_df_als["conf"] = (
    (train_df_als["have_like"] * 20.0)
    + (train_df_als["have_dislike"] * -10.0)
    + np.log1p(
        train_df_als["listen_count"] * (train_df_als["played_ratio_pct"] / 100.0)
    )
)

In [52]:
index, all_item_ids = create_index("data/filtered_embeddings.parquet", item_id_map)

added up to offset=100000, total indexed=51853
added up to offset=150000, total indexed=77962
added up to offset=200000, total indexed=103823
added up to offset=250000, total indexed=129605
added up to offset=300000, total indexed=155391
added up to offset=350000, total indexed=181244
added up to offset=400000, total indexed=207377
added up to offset=450000, total indexed=233406
added up to offset=500000, total indexed=259220
added up to offset=550000, total indexed=285054
added up to offset=600000, total indexed=311043
added up to offset=650000, total indexed=337165
added up to offset=700000, total indexed=363230
added up to offset=750000, total indexed=389039
added up to offset=800000, total indexed=415029
added up to offset=850000, total indexed=439962


In [9]:
hybrid = HybridModel()
hybrid.fit(train_df, df_merged, train_df_als)
hybrid.fit_ranker()

/opt/venv/lib/python3.11/site-packages/implicit/cpu/als.py:95: RuntimeWarning: OpenBLAS is configured to use 16 threads. It is highly recommended to disable its internal threadpool by setting the environment variable 'OPENBLAS_NUM_THREADS=1' or by calling 'threadpoolctl.threadpool_limits(1, "blas")'. Having OpenBLAS use a threadpool can lead to severe performance issues here.
  check_blas_config()


Prepere data
Train recomended models
trend_model


100%|██████████████████████████████████████████████████████████████████| 30/30 [00:20<00:00,  1.48it/s]


Prepere data for rancin model


100%|██████████████████████████████████████████████████████████████| 9219/9219 [19:49<00:00,  7.75it/s]


Train rancin model
0:	total: 925ms	remaining: 4m 36s
50:	total: 37.2s	remaining: 3m 1s
100:	total: 1m 12s	remaining: 2m 22s
150:	total: 1m 47s	remaining: 1m 46s
200:	total: 2m 26s	remaining: 1m 12s
250:	total: 3m 4s	remaining: 36s
299:	total: 3m 42s	remaining: 0us


In [ ]:
hybrid = HybridModel()
hybrid.fit(train_df, df_merged, train_df_als)

In [23]:
hybrid.hybrid_model = CatBoostRanker(
    iterations=300,
    learning_rate=0.3,
    depth=6,
    loss_function="YetiRank",
    # task_type="GPU",       # <<< ВКЛЮЧАЕТ GPU
    verbose=50,
)

hybrid.fit_ranker()

0:	total: 838ms	remaining: 4m 10s
50:	total: 37.5s	remaining: 3m 2s
100:	total: 1m 15s	remaining: 2m 29s
150:	total: 1m 52s	remaining: 1m 51s
200:	total: 2m 28s	remaining: 1m 13s
250:	total: 3m 5s	remaining: 36.1s
299:	total: 3m 40s	remaining: 0us


In [19]:
evaluate_model(hybrid, train_df, test_df)

ImportError: cannot import name 'get_gpu_device_count' from 'catboost' (/opt/venv/lib/python3.11/site-packages/catboost/__init__.py)

In [8]:
from tqdm import tqdm
from utils.metrics import ndcg_at_k, recall_at_k
import numpy as np
from utils.evaluate import filtering_listened_items, get_user_listened_items


def evaluate_model(model, train_df, test_df, k=10):
    test_df = test_df[test_df["event_type"]!="dislike"]
    grouped_users = test_df.groupby("uid")
    overall_recall = []
    overall_ndcg = []
    for uid, group in tqdm(grouped_users):
        # print(list(group["item_id"]))
        user_true = list(set(group["item_id"]))

        # print("-------------------")
        rec, weights = model.recommend(uid)
    
        # print(rec)
        user_listened_items = set(get_user_listened_items(train_df, uid))
        perfect = list(set(user_true)-set(user_listened_items))

        if len(perfect) == 0:
            continue   # skip user

        recall = recall_at_k(rec, perfect,  10)
        ndcg = ndcg_at_k(rec ,perfect,  10)

        # print(uid, ndcg, len(rec))

        
        overall_recall.append(recall)
        overall_ndcg.append(ndcg)
    
    print("Mean Recall@10:", np.mean(overall_recall))
    print("Mean NDCG@10:",np.mean(overall_ndcg))


# hybrid.limit = 100
# evaluate_model(hybrid, train_df, test_df)